<a href="https://colab.research.google.com/github/kmenesesrivera/codonusagebias/blob/main/3-Ajuste_de_Hiperparametros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Ajuste de Hiperparametros

##Lectura de Datos

In [3]:
import urllib.request
import zipfile, urllib.request, shutil
from __future__ import absolute_import, division, print_function, unicode_literals


url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00577/codon_usage.csv.zip'
filename = 'codon_usage.csv.zip'

In [4]:
with urllib.request.urlopen(url) as response, open(filename, 'wb') as out_file:
    shutil.copyfileobj(response, out_file)
    with zipfile.ZipFile(filename) as zf:
        zf.extractall()

In [5]:
!unzip codon_usage.csv

Archive:  codon_usage.csv
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
Archive:  codon_usage.csv.zip
replace codon_usage.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: Y
  inflating: codon_usage.csv         
replace __MACOSX/._codon_usage.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: Y
  inflating: __MACOSX/._codon_usage.csv  


In [6]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
labelencoder = LabelEncoder()

In [7]:
#Lectura del dataset
file_name = "codon_usage.csv"
dataset =  pd.read_csv('https://drive.google.com/uc?export=download&id=1Z4v43cvTwp920NyOdboDKP7_ytC_0tBC')

# Se observa que hay datos str que no permiten manipular los demás como numéricos.
dataset[['UUU', 'UUC']] = dataset[['UUU', 'UUC']].apply(pd.to_numeric, errors='coerce')
null_UUU = dataset['UUU'].isna().sum()
null_UUC= dataset['UUC'].isna().sum()
print ("Cantidad de datos nulos en codon UUU ",null_UUU)
print ("Cantidad de datos nuls en codon UUC ",null_UUC)


Cantidad de datos nulos en codon UUU  2
Cantidad de datos nuls en codon UUC  1


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
dataset

,Kingdom,DNAtype,SpeciesID,Ncodons,SpeciesName,UUU,UUC,UUA,UUG,CUU,CUC,CUA,CUG,AUU,AUC,AUA,AUG,GUU,GUC,GUA,GUG,GCU,GCC,GCA,GCG,CCU,CCC,CCA,CCG,UGG,GGU,GGC,GGA,GGG,UCU,UCC,UCA,UCG,AGU,AGC,ACU,ACC,ACA,ACG,UAU,UAC,CAA,CAG,AAU,AAC,UGU,UGC,CAU,CAC,AAA,AAG,CGU,CGC,CGA,CGG,AGA,AGG,GAU,GAC,GAA,GAG,UAA,UAG,UGA
0,vrl,0,100217,1995,Epizootic haematopoietic necrosis virus,0.01654,0.01203,0.00050,0.00351,0.01203,0.03208,0.00100,0.04010,0.00551,0.02005,0.00752,0.02506,0.01103,0.04110,0.00902,0.03308,0.01003,0.05013,0.01554,0.01103,0.02356,0.03208,0.01203,0.00501,0.01003,0.01203,0.03158,0.01905,0.02456,0.01353,0.02155,0.00251,0.00652,0.00150,0.01554,0.00501,0.02105,0.00902,0.01053,0.00501,0.02256,0.00301,0.03108,0.00401,0.02607,0.00251,0.01153,0.00501,0.02356,0.01053,0.03860,0.00401,0.00702,0.00401,0.00451,0.01303,0.03559,0.01003,0.04612,0.01203,0.04361,0.00251,0.00050,0.00000
1,vrl,0,100220,1474,Bohle iridovirus,0.02714,0.01357,0.00068,0.00678,0.00407,0.02849,0.00204,0.04410,0.01153,0.02510,0.00882,0.03324,0.00814,0.04071,0.00814,0.03256,0.01085,0.04885,0.01221,0.01357,0.00678,0.02714,0.01221,0.00407,0.01425,0.01221,0.01967,0.02239,0.01289,0.02103,0.01493,0.00407,0.00475,0.00068,0.02035,0.00950,0.02782,0.01425,0.00611,0.00475,0.02917,0.00407,0.02374,0.00882,0.02917,0.00271,0.01628,0.00204,0.01967,0.00543,0.03392,0.00136,0.00678,0.00136,0.00136,0.01696,0.03596,0.01221,0.04545,0.01560,0.04410,0.00271,0.00068,0.00000
2,vrl,0,100755,4862,Sweet potato leaf curl virus,0.01974,0.02180,0.01357,0.01543,0.00782,0.01111,0.01028,0.01193,0.02283,0.01604,0.01316,0.02180,0.01625,0.01872,0.01213,0.01070,0.02406,0.01234,0.01440,0.00514,0.01604,0.01460,0.02098,0.01070,0.01728,0.01851,0.00864,0.01172,0.01892,0.01933,0.01419,0.01296,0.00967,0.01337,0.01337,0.01851,0.01131,0.01419,0.01090,0.02612,0.01275,0.01522,0.02365,0.02962,0.01789,0.01625,0.01234,0.01604,0.01687,0.02077,0.03949,0.00864,0.00596,0.00926,0.00596,0.01974,0.02489,0.03126,0.02036,0.02242,0.02468,0.00391,0.00000,0.00144
3,vrl,0,100880,1915,Northern cereal mosaic virus,0.01775,0.02245,0.01619,0.00992,0.01567,0.01358,0.00940,0.01723,0.02402,0.02245,0.02507,0.02924,0.02089,0.02141,0.01723,0.01932,0.02141,0.00679,0.02245,0.00522,0.01358,0.00418,0.01410,0.00574,0.01201,0.00992,0.00366,0.02402,0.02663,0.02872,0.00992,0.02350,0.00522,0.01619,0.00836,0.02037,0.01358,0.02089,0.00731,0.02141,0.00888,0.01567,0.01253,0.02298,0.01358,0.00992,0.00888,0.00783,0.00679,0.03133,0.04282,0.00627,0.00261,0.00261,0.00366,0.01410,0.01671,0.03760,0.01932,0.03029,0.03446,0.00261,0.00157,0.00000
4,vrl,0,100887,22831,Soil-borne cereal mosaic virus,0.02816,0.01371,0.00767,0.03679,0.01380,0.00548,0.00473,0.02076,0.02716,0.00867,0.01310,0.02773,0.02803,0.00508,0.00920,0.02965,0.02878,0.00574,0.01572,0.01577,0.01007,0.00508,0.00604,0.00679,0.01205,0.03127,0.00775,0.00959,0.00797,0.02006,0.00359,0.00933,0.01191,0.01616,0.00788,0.02593,0.00854,0.01200,0.02098,0.02089,0.01367,0.01502,0.01809,0.02738,0.01796,0.01082,0.00705,0.01174,0.00858,0.03408,0.03964,0.00950,0.00429,0.00578,0.00604,0.01494,0.01734,0.04148,0.02483,0.03359,0.03679,0.00000,0.00044,0.00131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13023,pri,0,9601,1097,Pongo pygmaeus abelii,0.02552,0.03555,0.00547,0.01367,0.01276,0.02097,0.00820,0.03555,0.01459,0.03920,0.01003,0.02097,0.00912,0.02005,0.01003,0.03737,0.02279,0.01367,0.01094,0.00000,0.01641,0.01094,0.01185,0.00182,0.01094,0.00456,0.01276,0.01094,0.01276,0.02097,0.01003,0.01003,0.00091,0.01732,0.01459,0.01276,0.01276,0.01641,0.00820,0.02370,0.02097,0.02917,0.03464,0.02005,0.02005,0.01459,0.02461,0.01367,0.01459,0.01914,0.03008,0.00182,0.00547,0.00547,0.00820,0.01367,0.01094,0.01367,0.02279,0.02005,0.04102,0.00091,0.00091,0.00638
13024,pri,1,9601,2067,mitochondrion Pongo pygmaeus abe

In [9]:
dataset = dataset.dropna(subset=["UUU"])
dataset = dataset.dropna(subset=["UUC"])
 
null_UUU = dataset['UUU'].isna().sum()
null_UUC= dataset['UUC'].isna().sum()
print ("Cantidad de datos nulos en codon UUU ",null_UUU)
print ("Cantidad de datos nuls en codon UUC ",null_UUC)

dataset

Cantidad de datos nulos en codon UUU  0
Cantidad de datos nuls en codon UUC  0


,Kingdom,DNAtype,SpeciesID,Ncodons,SpeciesName,UUU,UUC,UUA,UUG,CUU,CUC,CUA,CUG,AUU,AUC,AUA,AUG,GUU,GUC,GUA,GUG,GCU,GCC,GCA,GCG,CCU,CCC,CCA,CCG,UGG,GGU,GGC,GGA,GGG,UCU,UCC,UCA,UCG,AGU,AGC,ACU,ACC,ACA,ACG,UAU,UAC,CAA,CAG,AAU,AAC,UGU,UGC,CAU,CAC,AAA,AAG,CGU,CGC,CGA,CGG,AGA,AGG,GAU,GAC,GAA,GAG,UAA,UAG,UGA
0,vrl,0,100217,1995,Epizootic haematopoietic necrosis virus,0.01654,0.01203,0.00050,0.00351,0.01203,0.03208,0.00100,0.04010,0.00551,0.02005,0.00752,0.02506,0.01103,0.04110,0.00902,0.03308,0.01003,0.05013,0.01554,0.01103,0.02356,0.03208,0.01203,0.00501,0.01003,0.01203,0.03158,0.01905,0.02456,0.01353,0.02155,0.00251,0.00652,0.00150,0.01554,0.00501,0.02105,0.00902,0.01053,0.00501,0.02256,0.00301,0.03108,0.00401,0.02607,0.00251,0.01153,0.00501,0.02356,0.01053,0.03860,0.00401,0.00702,0.00401,0.00451,0.01303,0.03559,0.01003,0.04612,0.01203,0.04361,0.00251,0.00050,0.00000
1,vrl,0,100220,1474,Bohle iridovirus,0.02714,0.01357,0.00068,0.00678,0.00407,0.02849,0.00204,0.04410,0.01153,0.02510,0.00882,0.03324,0.00814,0.04071,0.00814,0.03256,0.01085,0.04885,0.01221,0.01357,0.00678,0.02714,0.01221,0.00407,0.01425,0.01221,0.01967,0.02239,0.01289,0.02103,0.01493,0.00407,0.00475,0.00068,0.02035,0.00950,0.02782,0.01425,0.00611,0.00475,0.02917,0.00407,0.02374,0.00882,0.02917,0.00271,0.01628,0.00204,0.01967,0.00543,0.03392,0.00136,0.00678,0.00136,0.00136,0.01696,0.03596,0.01221,0.04545,0.01560,0.04410,0.00271,0.00068,0.00000
2,vrl,0,100755,4862,Sweet potato leaf curl virus,0.01974,0.02180,0.01357,0.01543,0.00782,0.01111,0.01028,0.01193,0.02283,0.01604,0.01316,0.02180,0.01625,0.01872,0.01213,0.01070,0.02406,0.01234,0.01440,0.00514,0.01604,0.01460,0.02098,0.01070,0.01728,0.01851,0.00864,0.01172,0.01892,0.01933,0.01419,0.01296,0.00967,0.01337,0.01337,0.01851,0.01131,0.01419,0.01090,0.02612,0.01275,0.01522,0.02365,0.02962,0.01789,0.01625,0.01234,0.01604,0.01687,0.02077,0.03949,0.00864,0.00596,0.00926,0.00596,0.01974,0.02489,0.03126,0.02036,0.02242,0.02468,0.00391,0.00000,0.00144
3,vrl,0,100880,1915,Northern cereal mosaic virus,0.01775,0.02245,0.01619,0.00992,0.01567,0.01358,0.00940,0.01723,0.02402,0.02245,0.02507,0.02924,0.02089,0.02141,0.01723,0.01932,0.02141,0.00679,0.02245,0.00522,0.01358,0.00418,0.01410,0.00574,0.01201,0.00992,0.00366,0.02402,0.02663,0.02872,0.00992,0.02350,0.00522,0.01619,0.00836,0.02037,0.01358,0.02089,0.00731,0.02141,0.00888,0.01567,0.01253,0.02298,0.01358,0.00992,0.00888,0.00783,0.00679,0.03133,0.04282,0.00627,0.00261,0.00261,0.00366,0.01410,0.01671,0.03760,0.01932,0.03029,0.03446,0.00261,0.00157,0.00000
4,vrl,0,100887,22831,Soil-borne cereal mosaic virus,0.02816,0.01371,0.00767,0.03679,0.01380,0.00548,0.00473,0.02076,0.02716,0.00867,0.01310,0.02773,0.02803,0.00508,0.00920,0.02965,0.02878,0.00574,0.01572,0.01577,0.01007,0.00508,0.00604,0.00679,0.01205,0.03127,0.00775,0.00959,0.00797,0.02006,0.00359,0.00933,0.01191,0.01616,0.00788,0.02593,0.00854,0.01200,0.02098,0.02089,0.01367,0.01502,0.01809,0.02738,0.01796,0.01082,0.00705,0.01174,0.00858,0.03408,0.03964,0.00950,0.00429,0.00578,0.00604,0.01494,0.01734,0.04148,0.02483,0.03359,0.03679,0.00000,0.00044,0.00131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13023,pri,0,9601,1097,Pongo pygmaeus abelii,0.02552,0.03555,0.00547,0.01367,0.01276,0.02097,0.00820,0.03555,0.01459,0.03920,0.01003,0.02097,0.00912,0.02005,0.01003,0.03737,0.02279,0.01367,0.01094,0.00000,0.01641,0.01094,0.01185,0.00182,0.01094,0.00456,0.01276,0.01094,0.01276,0.02097,0.01003,0.01003,0.00091,0.01732,0.01459,0.01276,0.01276,0.01641,0.00820,0.02370,0.02097,0.02917,0.03464,0.02005,0.02005,0.01459,0.02461,0.01367,0.01459,0.01914,0.03008,0.00182,0.00547,0.00547,0.00820,0.01367,0.01094,0.01367,0.02279,0.02005,0.04102,0.00091,0.00091,0.00638
13024,pri,1,9601,2067,mitochondrion Pongo pygmaeus abe

In [10]:
dataset.describe

<bound method NDFrame.describe of       Kingdom  DNAtype  SpeciesID   Ncodons  ...      GAG      UAA      UAG      UGA
0         vrl        0     100217      1995  ...  0.04361  0.00251  0.00050  0.00000
1         vrl        0     100220      1474  ...  0.04410  0.00271  0.00068  0.00000
2         vrl        0     100755      4862  ...  0.02468  0.00391  0.00000  0.00144
3         vrl        0     100880      1915  ...  0.03446  0.00261  0.00157  0.00000
4         vrl        0     100887     22831  ...  0.03679  0.00000  0.00044  0.00131
...       ...      ...        ...       ...  ...      ...      ...      ...      ...
13023     pri        0       9601      1097  ...  0.04102  0.00091  0.00091  0.00638
13024     pri        1       9601      2067  ...  0.00677  0.00242  0.00097  0.01887
13025     pri        1       9602      1686  ...  0.00297  0.00356  0.00119  0.02017
13026     pri        0       9606  40662582  ...  0.03959  0.00099  0.00079  0.00156
13027     pri        1       96

In [14]:
def preprocess_dataset(dataset, save_metadata=True):
  
  preprocessed_dataset = dataset.copy()

  #Determinamos aquellas variables que tengan un porcentaje de registros únicos por cada variable mayor al valor de 0.7
  threshold = 0.7

  unique_percentages = dataset.nunique() / len(dataset)

  criteria = unique_percentages > threshold

  columns_to_filter = unique_percentages[criteria].keys()
  
  # Está columna contiene datos descriptivos, por tanto será transformada a str.

  preprocessed_dataset['SpeciesName'] = preprocessed_dataset['SpeciesName'].astype(str)
  preprocessed_dataset['SpeciesName']
 
  #Eliminamos las variables seleccionadas en el paso anterior
  preprocessed_dataset.drop(columns_to_filter, axis=1, inplace=True)
 
  #Eliminamos los registros duplicados
  preprocessed_dataset.drop_duplicates(keep="first", inplace=True)

  #Determinamos que variables son del tipo númerico y cuales son categóricas
  numeric_columns = list()
  categorical_columns = list()
  dictionary_of_columns_with_index_to_categorical = dict()
  dictionary_of_columns_with_categorical_to_index = dict()

  for column in preprocessed_dataset:
    #Determinamos si la variable es numérica o no
    if pd.api.types.is_numeric_dtype(preprocessed_dataset[column]):
      numeric_columns.append(column)
    else:
      #Modificamos el tipo de dato de la variable mediante "astype"
      preprocessed_dataset[column] = preprocessed_dataset[column].astype("category")

      #Verificamos si el tipo de dato de la variable fue transformado a categórico correctamente
      if not pd.api.types.is_categorical_dtype(preprocessed_dataset[column]):
        raise Exception("La columna {} no se transformó correctamente a categórica".format(column))

      dictionary_of_columns_with_index_to_categorical[column] = dict()
      dictionary_of_columns_with_categorical_to_index[column] = dict()
      
      #Indexamos los valores (categorías), sin tomar en consideración los nulos, de la variable y guardamos esa información en los diccionarios
      for index, category in enumerate(preprocessed_dataset[column].cat.categories):
        dictionary_of_columns_with_index_to_categorical[column][index] = category
        dictionary_of_columns_with_categorical_to_index[column][category] = index
      
      categorical_columns.append(column)
   
  #Transformamos a números los valores (categorías) de las variables categóricas sin considerar los nulos
  preprocessed_dataset.replace(dictionary_of_columns_with_categorical_to_index, inplace=True)

  #Determinamos aquellas variables que tengan un porcentaje de registros nulos por cada variable mayor al valor de 0.7
  threshold = 0.7

  null_percentages = preprocessed_dataset[categorical_columns].isna().sum() / len(preprocessed_dataset)

  criteria = null_percentages > threshold

  columns_to_filter = null_percentages[criteria].keys()

  #Eliminamos las variables seleccionadas en el paso anterior
  preprocessed_dataset.drop(columns_to_filter, axis=1, inplace=True)

  #Eliminamos los registros duplicados
  preprocessed_dataset.drop_duplicates(keep="first", inplace=True)
 
  return preprocessed_dataset

In [15]:
preprocessed_dataset = preprocess_dataset(dataset)

In [16]:
X1=preprocessed_dataset.drop("Kingdom", axis=1)
X1=X1.drop("DNAtype", axis=1)
X2=X1
Y1 = preprocessed_dataset["Kingdom"]  
Y2 = preprocessed_dataset["DNAtype"] 

In [17]:
#Mediante el método "train_test_split" usaremos el 20% de la data para probar el modelo. El parámetro "random state" nos sirve para
#poder replicar la misma separación
x1_train, x1_test, y1_train, y1_test = train_test_split(X1, Y1, test_size=0.2, random_state=70)

In [18]:
#Mediante el método "train_test_split" usaremos el 20% de la data para probar el modelo. El parámetro "random state" nos sirve para
#poder replicar la misma separación
x2_train, x2_test, y2_train, y2_test = train_test_split(X2, Y2, test_size=0.2, random_state=70)

##Eliminas algunas caracteristicas 
Basandonos en el colab anterior decidimos usar solo 5 de las caracteristicas 

In [15]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif, mutual_info_classif

In [16]:
filtrado = SelectKBest(mutual_info_classif, k=5).fit(X1, Y1)
X_new = filtrado.transform(x1_train)
X_new[:5]

NameError: ignored

In [ ]:
Xt_new = filtrado.transform(x1_test )
Xt_new[:5]

##Entrenamos cada modelo

In [21]:
from sklearn.model_selection import GridSearchCV

###Decision Tree

#### y1

Decission Tree Base

In [72]:
dt1 = DecisionTreeClassifier()
dt1.fit(x1_train, y1_train)
print('Max Depth: ', dt1.tree_.max_depth)
print('Score: ',dt1.score(x1_test,y1_test))
print('F1: ', f1_score(y1_test, dt1.predict(x1_test), average='weighted'))

Max Depth:  29
Score:  0.7513471901462664
F1:  0.7509211543945985


Busqueda por Grilla

In [65]:
params = {"max_features": [65], "min_samples_leaf": [1,3,5,7], "min_samples_split": [2,4,6], "criterion": ["gini", "entropy"], "max_depth" : [32]}
tree = DecisionTreeClassifier()
dt_cv1 = GridSearchCV(tree, params, cv=5)
dt_cv1.fit(x1_train, y1_train)
print("Mejores Parametros GridSearchv: {}".format(dt_cv1.best_params_))
print("Mejor Score GridSearchv{}".format(dt_cv1.best_score_))   

Mejores Parametros GridSearchv: {'criterion': 'entropy', 'max_depth': 32, 'max_features': 65, 'min_samples_leaf': 1, 'min_samples_split': 2}
Mejor Score GridSearchv0.7636414560379913


Reentrenamos como los parametros encontrados

In [104]:
dt_final1 = DecisionTreeClassifier(max_depth=31, max_features=65, min_samples_leaf=1, min_samples_split=2 , criterion='entropy')
dt_final1.fit(x1_train, y1_train)
print('Score prueba: ',dt_final1.score(x1_test,y1_test))
print('F1: prueba', f1_score(y1_test, dt_final1.predict(x1_test), average='weighted'))

Score prueba:  0.7567359507313318
F1: prueba 0.7551751885355013


#### Y2

In [73]:
dt2 = DecisionTreeClassifier()
dt2.fit(x2_train, y2_train)
print('Max Depth: ', dt2.tree_.max_depth)
print('Score: ',dt2.score(x2_test,y2_test))
print('F1: ', f1_score(y2_test, dt2.predict(x2_test), average='weighted'))

Max Depth:  25
Score:  0.9668976135488837
F1:  0.9676489426382024


Busqueda por Grilla

In [74]:
params = {"max_features": [65], "min_samples_leaf": [1,3,5,7], "min_samples_split": [2,4,6], "criterion": ["gini", "entropy"], "max_depth" : [25]}
tree = DecisionTreeClassifier()
dt_cv2 = GridSearchCV(tree, params, cv=5)
dt_cv2.fit(x2_train, y2_train)
print("Mejores Parametros: {}".format(dt_cv2.best_params_))
print("Mejor Score {}".format(dt_cv2.best_score_))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


Mejores Parametros: {'criterion': 'gini', 'max_depth': 25, 'max_features': 65, 'min_samples_leaf': 5, 'min_samples_split': 2}
Mejor Score 0.9769992421580487


Reentrenamos como los parametros encontrados

In [100]:
dt_final2 = DecisionTreeClassifier(max_depth=11, max_features=65, min_samples_leaf=5, min_samples_split=2 , criterion='gini')
dt_final2.fit(x2_train, y2_train)
print('Score: ',dt_final2.score(x2_test,y2_test))
print('F1: ', f1_score(y2_test, dt_final2.predict(x2_test), average='weighted'))

Score:  0.9711316397228638
F1:  0.9704916595664262


###Random Forest

#### Y1

 
Random Forest Base

In [22]:
rt = RandomForestClassifier(40, n_jobs=-1, oob_score=True)
rt.fit(x1_train, y1_train)
print('Score: ',rt.score(x1_test,y1_test))
print('F1: ', f1_score(y1_test, rt.predict(x1_test), average='weighted')) 

KeyboardInterrupt: ignored

Busqueda por Grilla

In [109]:
params = {"n_estimators": [300], "max_depth": [30,40], "max_features": [65] , "min_samples_leaf": [1], "min_samples_split": [2]}
forest = RandomForestClassifier(oob_score=True)
rf_cv = GridSearchCV(forest, params, cv=5)
rf_cv.fit(x1_train, y1_train)
print("Mejores Parametros: {}".format(rf_cv.best_params_))
print("Mejor Score {}".format(rf_cv.best_score_))

Mejores Parametros: {'max_depth': 30, 'max_features': 65, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Mejor Score 0.8775857016472994


In [110]:
rf_final = RandomForestClassifier(oob_score=True, max_depth=30, max_features=65, min_samples_leaf=1, min_samples_split=2,n_estimators=300)
rf_final.fit(x1_train, y1_train)
print('Score: ',rf_final.score(x1_test,y1_test))
print('F1: ', f1_score(y1_test, rf_final.predict(x1_test), average='weighted'))

Score:  0.8729792147806005
F1:  0.8676773794275903


#### Y2

Random Forest Base

In [38]:
rt2 = RandomForestClassifier(40, n_jobs=-1, oob_score=True)
rt2.fit(x2_train, y2_train)
print('Score: ',rt2.score(x2_test,y2_test))
print('F1: ', f1_score(y2_test, rt.predict(x2_test), average='weighted')) 

Score:  0.9826789838337182
F1:  0.9813047842531513


Busqueda por Grilla

In [41]:
params2 = {"n_estimators": [ 200,400,600], "max_depth": [40,60,80], "max_features": [65] , "min_samples_leaf": [1], "min_samples_split": [2]}
forest2 = RandomForestClassifier(oob_score=True)
rf_cv2 = GridSearchCV(forest2, params2, cv=2)
rf_cv2.fit(x2_train, y2_train)
print("Mejores Parametros: {}".format(rf_cv2.best_params_))
print("Mejor Score {}".format(rf_cv2.best_score_))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  % (min_groups, self.n_splits)), UserWarning)


Mejores Parametros: {'max_depth': 60, 'max_features': 65, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 600}
Mejor Score 0.9837357306760734


In [43]:
rf_final = RandomForestClassifier(oob_score=True, max_depth=60, max_features=65, min_samples_leaf=1, min_samples_split=2,n_estimators=600)
rf_final.fit(x2_train, y2_train)
print('Score: ',rf_final.score(x2_test,y2_test))
print('F1: ', f1_score(y2_test, rf_final.predict(x2_test), average='weighted'))

Score:  0.9826789838337182
F1:  0.9814816903724841


###KNN

KNN BASE

#### Y1

In [47]:
knn = KNeighborsClassifier(3)
knn.fit(x1_train, y1_train)
print('Score: ',knn.score(x1_test,y1_test))
print('F1: ', f1_score(y1_test, knn.predict(x1_test), average='weighted'))

Score:  0.3394919168591224
F1:  0.3407297516068184


In [57]:
params = {"n_neighbors": [21,22,23,24], "weights": ['uniform', 'distance'], "metric": ['euclidean', 'manhattan']}
forest = KNeighborsClassifier()
knn_cv = GridSearchCV(forest, params, cv=3)
knn_cv.fit(x1_train, y1_train)
print("Mejores Parametros: {}".format(knn_cv.best_params_))
print("Mejor Score {}".format(knn_cv.best_score_))

Mejores Parametros: {'metric': 'manhattan', 'n_neighbors': 24, 'weights': 'distance'}
Mejor Score 0.37436297318379697


In [58]:
knn_final = KNeighborsClassifier(n_neighbors=24, metric='manhattan',weights='distance')
knn_final.fit(x1_train, y1_train)
print('Score: ',knn_final.score(x1_test,y1_test))
print('F1: ', f1_score(y1_test, knn_final.predict(x1_test), average='weighted'))

Score:  0.3695150115473441
F1:  0.3582960842286437


#### Y2

In [59]:
knn = KNeighborsClassifier(3)
knn.fit(x2_train, y2_train)
print('Score: ',knn.score(x2_test,y2_test))
print('F1: ', f1_score(y2_test, knn.predict(x2_test), average='weighted'))

Score:  0.76905311778291
F1:  0.7533254093162691


In [62]:
params = {"n_neighbors": [15,20,21,22,23,24,30], "weights": ['uniform', 'distance'], "metric": ['euclidean', 'manhattan']}
forest = KNeighborsClassifier()
knn_cv = GridSearchCV(forest, params, cv=3)
knn_cv.fit(x2_train, y2_train)
print("Mejores Parametros: {}".format(knn_cv.best_params_))
print("Mejor Score {}".format(knn_cv.best_score_))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)


Mejores Parametros: {'metric': 'manhattan', 'n_neighbors': 22, 'weights': 'distance'}
Mejor Score 0.7786548694579917


In [63]:
knn_final = KNeighborsClassifier(n_neighbors=22, metric='manhattan',weights='distance')
knn_final.fit(x2_train, y2_train)
print('Score: ',knn_final.score(x2_test,y2_test))
print('F1: ', f1_score(y2_test, knn_final.predict(x2_test), average='weighted'))

Score:  0.7848344880677445
F1:  0.7650454223833404


###XGBoost

#### Y1

In [64]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

#Requiere de 65 características según la selección de características 
param_grid = {
 'max_depth':range(9,10),
 'subsample': [0.7,0.9]
}

xgb_Model = XGBClassifier(learning_rate=0.1, n_estimators=80, min_child_weight=1, subsample= 0.7, nthread=-1, n_jobs=-1,scoring='f1')

xgb_Grid = GridSearchCV (estimator= xgb_Model, param_grid = param_grid )
xgb_Grid.fit(x1_train,y1_train)

print ("optimal max_depth",xgb_Grid.best_estimator_.max_depth) 
print ("optimal xgb_Grid.best_score_",xgb_Grid.best_score_)
  

optimal max_depth 9
optimal xgb_Grid.best_score_ 0.9084779691131952


In [65]:
xgb_Grid.score(x1_test, y1_test)

0.9114703618167821

#### Y2

In [66]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

#Requiere de 65 características según la selección de características 

param_grid = {
 'max_depth':range(9,10)
}

xgb_Model2 = XGBClassifier(learning_rate=0.01, n_estimators=80, min_child_weight=2, gamma=0, colsample_bytree=0.8, subsample= 0.7, objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27, n_jobs=-1,scoring='roc_auc')
xgb_Grid2 = GridSearchCV (estimator= xgb_Model2, param_grid = param_grid )
 
xgb_Grid2.fit(x2_train,y2_train)
 
print ("optimal max_depth",xgb_Grid2.best_estimator_.max_depth)
print ("optimal xgb_Grid2.best_score_",xgb_Grid2.best_score_)
 

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


optimal max_depth 9
optimal xgb_Grid2.best_score_ 0.9857567841205956


In [67]:
xgb_Grid2.score(x2_test, y2_test)

0.9792147806004619

### Ligth GBM

In [68]:
import lightgbm as lgb 

#### Y1

In [69]:
#Declaración de parámetros para el modelo


params = {'boosting_type': 'gbdt',
          'max_depth' : 10,
          'nthread': -1, 
          'max_bin': 512,
          'subsample_for_bin': 200,
          'subsample_freq': 1,
          'min_split_gain': 0.5,
          'min_child_weight': 2,
          'min_child_samples': 6,
          'scale_pos_weight': 1,
          'num_class' : 1,
          'metric' : 'binary_error'}

#Declaración de parámetros para GridSearchv

gridParams = {
    'learning_rate': [0.1],
    'n_estimators': [91],
    'num_leaves': [19],
    'objective' : ['binary'],
    'random_state' : [30], 
    'colsample_bytree' : [0.65],
    'subsample' : [1],
    'reg_alpha' : [1],
    'reg_lambda' : [1.2],
    }

#Requiere de 64 características según la selección de características 

#Declaración de Modelo
mdl = lgb.LGBMClassifier(boosting_type= 'gbdt',
          objective = 'binary',
          n_jobs = -1, 
          silent = True,
          max_depth = params['max_depth'],
          max_bin = params['max_bin'],
          subsample_for_bin = params['subsample_for_bin'],
          subsample_freq = params['subsample_freq'],
          min_split_gain = params['min_split_gain'],
          min_child_weight = params['min_child_weight'],
          min_child_samples = params['min_child_samples'],
          scale_pos_weight = params['scale_pos_weight'])

mdl.get_params().keys()
#Ejecuciòn de GridSearchv
grid = GridSearchCV(mdl, gridParams,
                    verbose=0,
                    cv=4,
                    n_jobs=-1)
#Entrenamiento
grid.fit(x1_train, y1_train)

GridSearchCV(cv=4, error_score=nan,
             estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None,
                                      colsample_bytree=1.0,
                                      importance_type='split',
                                      learning_rate=0.1, max_bin=512,
                                      max_depth=10, min_child_samples=6,
                                      min_child_weight=2, min_split_gain=0.5,
                                      n_estimators=100, n_jobs=-1,
                                      num_leaves=31, objective='binary',
                                      random_state=None, reg_alpha=0.0,
                                      reg_lambda=0.0...
                                      silent=True, subsample=1.0,
                                      subsample_for_bin=200, subsample_freq=1),
             iid='deprecated', n_jobs=-1,
             param_grid={'colsample_bytree': [0.65], 'learning_rate': [0.1],
        

In [70]:
#Scores
print(grid.best_params_)
print(grid.best_score_)
print(grid.score(x1_test, y1_test))

{'colsample_bytree': 0.65, 'learning_rate': 0.1, 'n_estimators': 91, 'num_leaves': 19, 'objective': 'binary', 'random_state': 30, 'reg_alpha': 1, 'reg_lambda': 1.2, 'subsample': 1}
0.8991437090762926
0.9026173979984603


#### Y2

In [71]:
#Declaración de parámetros para el modelo
params2 = {'boosting_type': 'gbdt',
          'max_depth' : 6,
          'objective': 'binary',
          'nthread': -1,
          'num_leaves': 64,
          'learning_rate': 0.05,
          'max_bin': 512,
          'subsample_for_bin': 200,
          'subsample': 1,
          'subsample_freq': 1,
          'colsample_bytree': 0.8,
          'reg_alpha': 5,
          'reg_lambda': 10,
          'min_split_gain': 0.5,
          'min_child_weight': 2,
          'min_child_samples': 6,
          'scale_pos_weight': 1,
          'num_class' : 1,
          'metric' : 'binary_error'}
          
#Declaración de parámetros para GridSearchv

gridParams2 = {
    'learning_rate': [0.1],
    'n_estimators': [90],
    'num_leaves': [16],
    'boosting_type' : ['gbdt'],
    'objective' : ['binary'],
    'random_state' : [30],
    'colsample_bytree' : [0.65],
    'subsample' : [0.9],
    'reg_alpha' : [1],
    'reg_lambda' : [1],
    'max_depth' : [8]
    }

#Requiere de 64 características según la selección de características 

#Declaración de Modelo
mdl2 = lgb.LGBMClassifier(boosting_type= 'gbdt',
          objective = 'binary',
          n_jobs = -1, 
          silent = True,
          max_depth = params2['max_depth'],
          max_bin = params2['max_bin'],
          subsample_for_bin = params2['subsample_for_bin'],
          subsample = params2['subsample'],
          subsample_freq = params2['subsample_freq'],
          min_split_gain = params2['min_split_gain'],
          min_child_weight = params2['min_child_weight'],
          min_child_samples = params2['min_child_samples'],
          scale_pos_weight = params2['scale_pos_weight'])

mdl2.get_params().keys()

#Ejecuciòn de GridSearchv
grid2 = GridSearchCV(mdl2, gridParams2,
                    verbose=0,
                    cv=4,
                    n_jobs=-1)

#Entrenamiento
grid2.fit(x2_train, y2_train)

#Scores
print(grid2.best_params_)
print(grid2.best_score_) 
print(grid2.score(x2_test, y2_test))


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  % (min_groups, self.n_splits)), UserWarning)


{'boosting_type': 'gbdt', 'colsample_bytree': 0.65, 'learning_rate': 0.1, 'max_depth': 8, 'n_estimators': 90, 'num_leaves': 16, 'objective': 'binary', 'random_state': 30, 'reg_alpha': 1, 'reg_lambda': 1, 'subsample': 0.9}
0.9902800960900286
0.9869130100076983
